# Compare Transit Skims

In [1]:
import os
import pandas as pd
import numpy as np
from functools import reduce
import openmatrix as omx 

### Remote I/O

In [2]:
legacy_dir = os.path.join("C:/", "projects", "met_council", "model_files", "outputs2015", "outputs")
new_dir = os.path.join("D:/", "david_ory_github", "client_met_council_network", "data", "test_new_network_outputs")
compare_dir = os.path.join("D:/", "david_ory_github", "client_met_council_network", "data", "interim")

out_filename = os.path.join(compare_dir, "compare-transit.csv")

### Start with walk transit

In [3]:
filename = os.path.join(legacy_dir, "XIT_WK_SKIM_4_PK.OMX")
legacy_omx = omx.open_file(filename)

filename = os.path.join(new_dir, "XIT_WK_SKIM_4_PK.OMX")
new_omx = omx.open_file(filename)

filename = os.path.join(legacy_dir, "XIT_TRIP_4_PK_CLEAN.OMX")
demand_omx = omx.open_file(filename)

In [4]:
legacy_omx.list_matrices()

['FARE',
 'FAREP',
 'FARER',
 'FARES',
 'IWAIT',
 'TRNTIME',
 'WLKACC',
 'WLKEGR',
 'WLKXFER',
 'XFERS',
 'XWAIT']

In [5]:
new_omx.list_matrices()

['FARE',
 'FAREP',
 'FARER',
 'FARES',
 'IWAIT',
 'TRNTIME',
 'WLKACC',
 'WLKEGR',
 'WLKXFER',
 'XFERS',
 'XWAIT']

In [6]:
demand_omx.list_matrices()

['DriveToTransit', 'WalkToTransit']

In [7]:
def make_dataframe(input_mtx, column_name):
    df = pd.DataFrame(input_mtx)
    df = df.add_prefix('dest_')
    df['orig'] = df.index
    df = pd.wide_to_long(df, 'dest_', 'orig', 'dest')
    df = df.reset_index().rename(columns = {'dest_': column_name})
    df['orig'] = df['orig'] + 1
    df['dest'] = df['dest'] + 1
    
    return df

In [8]:
leg_ivtt_df = make_dataframe(legacy_omx['TRNTIME'], 'legacy_ivtt')
leg_iwait_df = make_dataframe(legacy_omx['IWAIT'], 'legacy_iwait')
leg_xwait_df = make_dataframe(legacy_omx['XWAIT'], 'legacy_xwait')
leg_wlkacc_df = make_dataframe(legacy_omx['WLKACC'], 'legacy_wlkacc')
leg_wlkegr_df = make_dataframe(legacy_omx['WLKEGR'], 'legacy_wlkegr')
leg_wlkxfer_df = make_dataframe(legacy_omx['WLKXFER'], 'legacy_wlkxfer')
leg_xfers_df = make_dataframe(legacy_omx['XFERS'], 'legacy_xfers')

In [9]:
new_ivtt_df = make_dataframe(new_omx['TRNTIME'], 'new_ivtt')
new_iwait_df = make_dataframe(new_omx['IWAIT'], 'new_iwait')
new_xwait_df = make_dataframe(new_omx['XWAIT'], 'new_xwait')
new_wlkacc_df = make_dataframe(new_omx['WLKACC'], 'new_wlkacc')
new_wlkegr_df = make_dataframe(new_omx['WLKEGR'], 'new_wlkegr')
new_wlkxfer_df = make_dataframe(new_omx['WLKXFER'], 'new_wlkxfer')
new_xfers_df = make_dataframe(new_omx['XFERS'], 'new_xfers')

In [10]:
walk_df = make_dataframe(demand_omx['WalkToTransit'], 'walk')
drive_df = make_dataframe(demand_omx['DriveToTransit'], 'drive')

In [11]:
df_list = [leg_iwait_df, leg_xwait_df, leg_wlkacc_df, leg_wlkegr_df, leg_wlkxfer_df]
leg_ovtt_df = reduce(lambda left,right: pd.merge(left,right,on=["orig", "dest"]), df_list)

In [12]:
leg_ovtt_df["legacy_ovtt"] = leg_ovtt_df["legacy_iwait"] + leg_ovtt_df["legacy_xwait"] + leg_ovtt_df["legacy_wlkacc"] + leg_ovtt_df["legacy_wlkegr"] + leg_ovtt_df["legacy_wlkxfer"]
                
leg_ovtt_df = leg_ovtt_df[leg_ovtt_df["legacy_iwait"] > 0]            
leg_ovtt_df = leg_ovtt_df[["orig", "dest", "legacy_ovtt"]]
leg_ovtt_df.head()

,orig,dest,legacy_ovtt
165360,67,55,60.08
165361,68,55,56.96
165362,69,55,59.19
165363,70,55,59.06
165364,71,55,82.70


In [13]:
df_list = [new_iwait_df, new_xwait_df, new_wlkacc_df, new_wlkegr_df, new_wlkxfer_df]
new_ovtt_df = reduce(lambda left,right: pd.merge(left,right,on=["orig", "dest"]), df_list)

In [14]:
new_ovtt_df["new_ovtt"] = new_ovtt_df["new_iwait"] + new_ovtt_df["new_xwait"] + new_ovtt_df["new_wlkacc"] + new_ovtt_df["new_wlkegr"] + new_ovtt_df["new_wlkxfer"]
                
new_ovtt_df = new_ovtt_df[new_ovtt_df["new_iwait"] > 0]            
new_ovtt_df = new_ovtt_df[["orig", "dest", "new_ovtt"]]
new_ovtt_df.head()

,orig,dest,new_ovtt
162287,55,54,67.79
162288,56,54,89.17
162289,57,54,104.33
162296,64,54,91.00
162297,65,54,97.59


In [15]:
demand_df = pd.merge(
   walk_df,
   drive_df,
   how = "inner",
   on = ["orig", "dest"]
)

demand_df["demand"] = demand_df["walk"] + demand_df["drive"]
demand_df = demand_df[["orig", "dest", "demand"]]
demand_df.head()

,orig,dest,demand
0,1,1,0.0
1,2,1,0.0
2,3,1,0.0
3,4,1,0.0
4,5,1,0.0


In [16]:
df_list = [leg_ivtt_df, new_ivtt_df, leg_ovtt_df, new_ovtt_df, leg_xfers_df, new_xfers_df, demand_df]
t_df = reduce(lambda left, right: pd.merge(left, right, how="outer", on=["orig", "dest"]), df_list)
t_df.head()

,orig,dest,legacy_ivtt,new_ivtt,legacy_ovtt,new_ovtt,legacy_xfers,new_xfers,demand
0,1,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,2,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
2,3,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
3,4,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
4,5,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0


In [17]:
t_df.head()

,orig,dest,legacy_ivtt,new_ivtt,legacy_ovtt,new_ovtt,legacy_xfers,new_xfers,demand
0,1,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,2,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
2,3,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
3,4,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0
4,5,1,0.0,0.0,NaN,NaN,0.0,0.0,0.0


In [18]:
out_df = t_df[(t_df['legacy_ivtt'] !=0) | (t_df['new_ivtt'] !=0)]

In [19]:
out_df.shape

(3128176, 9)

In [20]:
t_df.shape

(9369721, 9)

In [21]:
out_df.to_csv(out_filename, index = False)